In [3]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

## Trabalho 3
* Suponha que há duas salas de aula. A sala A comporta 40 alunos e a sala B comporta 20 alunos
* Deve-se alocar 4 turmas nestas duas salas, nos turnos matutino e vespertino conforme a tabela abaixo

| Turma | Quantidade de Alunos |
|-------|----------------------|
| T1    | 30                   |
| T2    | 15                   |
| T3    | 18                   |
| T4    | 20                   |

#### Restrições
* O professor Romeu leciona para as turmas T1 e T3 e prefere usar sempre a mesma sala
* A professora Julieta leciona para a turma T2 e não pode trabalhar pela manhã
* A professora Silvana leciona para a turma T4, que tem alunos em comum com a turma T2
* Além das restrições consideradas acima, devemos lembrar que um professor não pode estar em duas salas diferentes no mesmo horário.

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import random
from collections import defaultdict

from ipywidgets import *
import ipywidgets as widgets

from IPython.display import display, HTML

class Individuo():   
    
    def __init__(self, geracao=1, **kwargs):
        if geracao == 1:
            self.gera_cromossomo()
        self.geracao = geracao
        self.aptidao = 0
                        
    def __repr__(self): # formata representacao do objeto quanto utiliza print/display
        return np.array2string(self.cromossomo)
    
    def gera_cromossomo(self):
        self.cromossomo = np.arange(1,5)
        np.random.shuffle(self.cromossomo)
        
    
    def avaliacao(self):
        self.aptidao = 10
        turmas = {}
        # criar dicionario 'turmas' como chave:valor sendo -> turma:indice
        for i in range(len(self.cromossomo)):
            turmas[self.cromossomo[i]] = i
        
        # A sala A (idx: 0 e 2) comporta 40 alunos e a sala B (idx 1 e 3)comporta 20 alunos
        # Turma 1 eh a unica com quantidade de alunos (30) maior que a capacidade da sala B (20 alunos)
        if(turmas[1] == 1 or turmas[1] == 3):
            self.aptidao -= 1
        else:
            self.aptidao += 1
        
        # O professor Romeu leciona para as turmas T1 e T3 e prefere usar sempre a mesma sala
        if (turmas[1] % 2) == (turmas[3] % 2):
            self.aptidao +=  1
        else:
            self.aptidao -=  1
            
        # A professora Julieta leciona para a turma T2 e não pode trabalhar pela manhã.
        if turmas[2] < 2:
            self.aptidao -= 1
        else:
            self.aptidao += 1
            
        # A professora Silvana leciona para a turma T4, que tem alunos em comum com a turma T2.
        if (turmas[4] < 2 and turmas[2] < 2) or (turmas[4] > 1 and turmas[2] > 1):
            self.aptidao -= 1
        else:
            self.aptidao += 1
        
        # Um professor não pode estar em duas salas diferentes no mesmo horário
        # No caso somente o professor Romeu tem aula com duas turmas
        if (turmas[1] < 2 and turmas[3] < 2) or (turmas[1] > 1 and turmas[3] > 1):
            self.aptidao -= 1
        else:
            self.aptidao += 1
                
        
    def cruzamento(self, outro_individuo):
        f1_cromossomo = np.concatenate((self.cromossomo[0:2], outro_individuo.cromossomo[2::]))
        f2_cromossomo = np.concatenate((outro_individuo.cromossomo[0:2], self.cromossomo[2::]))
        
        filhos = [f1_cromossomo, f2_cromossomo]
        
        # substitui os valores repetidos pelos valores faltantes
        for f in filhos:
            qtd = {}
            qtd = defaultdict(lambda:0, qtd) # inicializa todos valores com 0 default
            for i in range(len(f)):
                if qtd[f[i]] > 0:
                    qtd[f[i]] += 1
                    f[i] = 5
                else:
                    qtd[f[i]] += 1
                    
            faltando = []
            for i in range(1,5):
                if qtd[i] == 0:
                    faltando.append(i)
                    
            for i in range(len(f)):
                if f[i] == 5:
                    f[i] = faltando.pop()
                    
        filhos_individuos = [Individuo(geracao=self.geracao+1),
                 Individuo(geracao=self.geracao+1)]
        filhos_individuos[0].cromossomo = filhos[0]
        filhos_individuos[1].cromossomo = filhos[1]
        
        return filhos_individuos
    
    def mutacao(self, taxa_mutacao):
        if random.random() < taxa_mutacao: 
            idxs = random.sample(range(0,4), 2)
            aux = self.cromossomo[idxs[0]]
            self.cromossomo[idxs[0]] = self.cromossomo[idxs[1]]
            self.cromossomo[idxs[1]] = aux
        return self
    

class AlgoritmoGenetico():
    
    def __init__(self, tam_populacao, taxa_cruzamento, taxa_mutacao,
                qtd_geracoes, selecao, elitismo, tam_torneio=0, **kwargs):
        self.tam_populacao = tam_populacao
        self.populacao = np.array([])
        self.taxa_cruzamento = taxa_cruzamento
        self.taxa_mutacao = taxa_mutacao
        self.qtd_geracoes = qtd_geracoes
        self.geracao = 0
        self.selecao = selecao
        self.tam_torneio = tam_torneio
        self.elitismo = elitismo
        self.lista_solucao = []
        self.melhor_solucao = 0
    
    def inicializa_populacao(self):   
        lista = []
        for i in range(self.tam_populacao):
            lista.append(Individuo())
        self.populacao = np.array(lista)
        self.melhor_solucao = self.populacao[0]    
        
    def ordena_populacao(self):
        self.populacao = sorted(self.populacao,
                               key =  lambda individuo: individuo.aptidao,
                               reverse = True)
        self.populacao = np.array(self.populacao)
        
    def calcula_aptidao(self):
        for ind in self.populacao:
            ind.avaliacao()
    
    def melhor_individuo(self, individuo):
        if individuo.aptidao > self.melhor_solucao.aptidao:
            self.melhor_solucao = individuo
    
    def soma_aptidoes(self):
        soma = 0
        for i in self.populacao:
            soma += i.aptidao
        return soma
            
    def seleciona_pais(self, soma_aptidoes=0):
        '''
        retorna um array com o par de pais com base no tipo de selecao
        '''
        if self.selecao == 'roleta':
            r_pais = [random.random() * soma_aptidoes]
            while True:
                r = random.random() * soma_aptidoes
                if r not in r_pais:
                    r_pais.append(r)
                    break
                    
            pais = []
            for r in r_pais:
                pai = -1
                i = 0
                soma = 0
                while i < len(self.populacao) and soma < r:
                    soma += self.populacao[i].aptidao
                    pai += 1
                    i += 1
                pais.append(self.populacao[pai])
            return pais
        
        elif self.selecao == 'torneio':
            indices_torneio = random.sample(range(0, self.tam_populacao), self.tam_torneio)
            torneio = self.populacao[indices_torneio]
            torneio = sorted(torneio, key=lambda i: i.aptidao, reverse=True)
            return [torneio[0], torneio[1]]
        
    
    def seleciona_elitismo(self):
        '''
        retorna lista contendo os individuos elitistas
        '''
        self.ordena_populacao()
        return self.populacao[0:self.elitismo]
    
    

########## entradas dos parametros ##########

print('\n\nTamanho da população:')
def f(populacao=50):
    return populacao
populacao = interactive(f, populacao=(10, 100, 10))
display(populacao)

print('\n\nProbabilidade de cruzamento:')
def f(cruzamento=95.0):
    return cruzamento
cruzamento = interactive(f, cruzamento=(5.0, 95.0, 0.5))
display(cruzamento)

print('\n\nProbabilidade de mutação:')
def f(mutacao=0.1):
    return mutacao
mutacao = interactive(f, mutacao=(1.0, 10.0, 0.5))
display(mutacao)

print('\n\nQuantidade de gerações:')
def f(geracoes=30):
    return geracoes
geracoes = interactive(f, geracoes=(10, 100))
display(geracoes)

print('\n\nMétodo de seleção:')
def f(selecao):
    return selecao
selecao = interactive(f, selecao=['Roleta', 'Torneio'])
display(selecao)

print("""\n\nTamanho do Torneio:\n *ignorar caso o 'Método de seleção' selecionado seja diferente de 'Torneio'*""")
def f(tam_torneio=10):
    return tam_torneio
tam_torneio = interactive(f, tam_torneio=(2, 50, 2))
display(tam_torneio)

print('\n\nTamanho do elitismo:')
def f(elitismo=1):
    return elitismo
elitismo = interactive(f, elitismo=(0, 2))
display(elitismo)

########## processamento ##########
def processar():
    
    alg = AlgoritmoGenetico(
        tam_populacao = populacao.result,
        taxa_cruzamento = cruzamento.result/100.0,
        taxa_mutacao = mutacao.result/100.0,
        qtd_geracoes = geracoes.result,
        selecao = selecao.result.lower(),
        elitismo = elitismo.result,
        tam_torneio = tam_torneio.result)
    
    ## inicio ##
    
    alg.inicializa_populacao()
    
    graf_aptidoes = []
    graf_geracoes = []    
    
    for g in range(0, alg.qtd_geracoes):
        graf_geracoes.append(g)        
        
        alg.calcula_aptidao()
        alg.ordena_populacao()
        
        nova_popu = []
        soma_apt = alg.soma_aptidoes() if alg.selecao == 'roleta' else 0
        for c in range(0, alg.tam_populacao, 2):
            pais = alg.seleciona_pais(soma_apt)
            if random.random() <= alg.taxa_cruzamento:
                filhos = pais[0].cruzamento(pais[1])
                for filho in filhos:
                    nova_popu.append(filho.mutacao(alg.taxa_mutacao))
            else:
                for pai in pais:
                    nova_popu.append(pai)
        
        elitistas = alg.seleciona_elitismo()
        indices = random.sample(range(0, alg.tam_populacao), len(elitistas))
        for idx in range(0, len(elitistas)):
            alg.populacao[indices[idx]] = elitistas[idx]
            
        alg.populacao = np.array(nova_popu)
        
        alg.calcula_aptidao()
        alg.ordena_populacao()
        alg.lista_solucao.append(alg.populacao[0])
        alg.melhor_individuo(alg.populacao[0])
          
        string = "<h4><strong>Geração:</strong> {:}</h4>".format(g+1)
        display(HTML(string))          
        
        graf_aptidoes.append(alg.populacao[0].aptidao)        
        
        t = "T"                
        dataframe = np.array([['','Sala A','Sala B'],
                ['Matutino',t+str(alg.populacao[0].cromossomo[0]),t+str(alg.populacao[0].cromossomo[1])],
                ['Vespertino',t+str(alg.populacao[0].cromossomo[2]),t+str(alg.populacao[0].cromossomo[3])]])
        
        display(pd.DataFrame(data=dataframe[1:,1:],
                  index=dataframe[1:,0],
                  columns=dataframe[0,1:]))
        
        print('-------------------------')

    string = "<h4><strong>Geração da melhor solução:</strong> {:}</h4>".format(alg.melhor_solucao.geracao)
    display(HTML(string))
    
    string = "<h4><strong>Melhor solução:</strong></h4>"
    display(HTML(string))
    
    dataframe = np.array([['','Sala A','Sala B'],
                ['Matutino',t+str(alg.melhor_solucao.cromossomo[0]),t+str(alg.melhor_solucao.cromossomo[1])],
                ['Vespertino',t+str(alg.melhor_solucao.cromossomo[2]),t+str(alg.melhor_solucao.cromossomo[3])]])
        
    display(pd.DataFrame(data=dataframe[1:,1:],
                  index=dataframe[1:,0],
                  columns=dataframe[0,1:]))
    
    
    N = 1000
    random_x = np.random.randn(N)
    random_y = np.random.randn(N)
    
    l = plt.plot(graf_geracoes, graf_aptidoes, 'ro')
    plt.setp(l, markersize=15)
    plt.setp(l, markerfacecolor='C0')

    string = "<h4><strong>Gráfico das aptidões:</strong></h4>"
    display(HTML(string))    
    plt.show()
    
widgets.interact_manual.opts['manual_name'] = 'Processar algoritmo' # muda texto do botao
interact_manual(processar); # metodo a executar quando pressionar o botao



Tamanho da população:


interactive(children=(IntSlider(value=50, description='populacao', min=10, step=10), Output()), _dom_classes=(…



Probabilidade de cruzamento:


interactive(children=(FloatSlider(value=95.0, description='cruzamento', max=95.0, min=5.0, step=0.5), Output()…



Probabilidade de mutação:


...



Tamanho do Torneio:
 *ignorar caso o 'Método de seleção' selecionado seja diferente de 'Torneio'*


interactive(children=(IntSlider(value=10, description='tam_torneio', max=50, min=2, step=2), Output()), _dom_c…



Tamanho do elitismo:


interactive(children=(IntSlider(value=1, description='elitismo', max=2), Output()), _dom_classes=('widget-inte…

interactive(children=(Button(description='Processar algoritmo', style=ButtonStyle()), Output()), _dom_classes=…